In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import cv2

In [5]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Define paths
base_dir = 'data'
happy_dir = os.path.join(base_dir, 'Happy')
sad_dir = os.path.join(base_dir, 'Sad')
angry_dir = os.path.join(base_dir, 'Angry')

# Collect image paths and labels
image_paths = []
labels = []

# Load Happy images
for img_file in os.listdir(happy_dir):
    if img_file.endswith(('.jpg', '.png', '.jpeg')):
        image_paths.append(os.path.join(happy_dir, img_file))
        labels.append(0)  # 0 for Happy

# Load Sad images
for img_file in os.listdir(sad_dir):
    if img_file.endswith(('.jpg', '.png', '.jpeg')):
        image_paths.append(os.path.join(sad_dir, img_file))
        labels.append(1)  # 1 for Sad

# Load Angry images
for img_file in os.listdir(angry_dir):
    if img_file.endswith(('.jpg', '.png', '.jpeg')):
        image_paths.append(os.path.join(angry_dir, img_file))
        labels.append(2)  # 2 for Angry

# Convert labels to categorical (one-hot encoding)
labels = to_categorical(labels, num_classes=3)

# Define image size (resize for consistency)
IMG_SIZE = 128
data = []

for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is None:
        print(f"Warning: Couldn't read image {img_path}")
        continue
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize images
    img = img / 255.0  # Normalize pixel values (0 to 1)
    data.append(img)

# Convert lists to NumPy arrays
data = np.array(data)

# Ensure labels match the length of data
labels = labels[:len(data)]

# Split dataset into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

print(f"Train shape: {X_train.shape}, {y_train.shape}")
print(f"Validation shape: {X_val.shape}, {y_val.shape}")


Train shape: (211, 128, 128, 3), (211, 3)
Validation shape: (53, 128, 128, 3), (53, 3)


#### CNN Model Training

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Reduces overfitting
        Dense(3, activation='softmax')  # 3 classes: Happy, Sad, Angry
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


1. Train CNN Without Augmentation

In [7]:
# Create the model
model = create_cnn()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

C:\Users\kiplimo\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - accuracy: 0.3062 - loss: 2.3715 - val_accuracy: 0.3396 - val_loss: 1.2827
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - accuracy: 0.3745 - loss: 1.1272 - val_accuracy: 0.5283 - val_loss: 1.0699
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.5040 - loss: 0.9978 - val_accuracy: 0.3962 - val_loss: 1.0530
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.6747 - loss: 0.8480 - val_accuracy: 0.4906 - val_loss: 1.0394
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - accuracy: 0.6780 - loss: 0.6987 - val_accuracy: 0.6226 - val_loss: 0.9096
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.9062 - loss: 0.4634 - val_accuracy: 0.5660 - val_loss: 0.9815
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.8847 - loss: 0.3410 - val_accuracy: 0.5472 - val_loss: 1.0490
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - accuracy: 0.9659 - loss: 0.1957 - val_accuracy: 0.5660 - val_loss:

2. Train the CNN with Augmentation

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define augmentation
datagen = ImageDataGenerator(
    rotation_range=20,        # Rotate images randomly by up to 20 degrees
    width_shift_range=0.2,    # Shift width by 20%
    height_shift_range=0.2,   # Shift height by 20%
    shear_range=0.2,          # Shear transformation
    zoom_range=0.2,           # Zoom in/out
    horizontal_flip=True,     # Flip images horizontally
    fill_mode='nearest'       # Fill missing pixels after transformation
)

# Train with augmentation
augmented_model = create_cnn()
augmented_model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    epochs=10)


C:\Users\kiplimo\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\kiplimo\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 236ms/step - accuracy: 0.3458 - loss: 1.7292 - val_accuracy: 0.3962 - val_loss: 1.0508
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 207ms/step - accuracy: 0.4425 - loss: 1.0602 - val_accuracy: 0.3396 - val_loss: 1.0260
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step - accuracy: 0.4015 - loss: 1.0371 - val_accuracy: 0.3396 - val_loss: 1.0021
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 207ms/step - accuracy: 0.4082 - loss: 1.0308 - val_accuracy: 0.3962 - val_loss: 1.0326
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - accuracy: 0.3993 - loss: 1.0114 - val_accuracy: 0.3962 - val_loss: 0.9928
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step - accuracy: 0.4732 - loss: 0.9702 - val_accuracy: 0.5660 - val_loss: 0.9772
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step - accuracy: 0.5565 - loss: 0.9820 - val_accuracy: 0.5472 - val_loss: 0.9799
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 214ms/step - accuracy: 0.4448 - loss: 1.0519 - val_accuracy: 0.4717 - val_loss:

In [10]:
original_acc = model.evaluate(X_val, y_val, verbose=0)[1]
augmented_acc = augmented_model.evaluate(X_val, y_val, verbose=0)[1]

print(f'Original Model Accuracy: {original_acc:.2f}')
print(f'Augmented Model Accuracy: {augmented_acc:.2f}')


Original Model Accuracy: 0.60
Augmented Model Accuracy: 0.64
